In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain.schema import SystemMessage

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

c:\Users\KIM\Desktop\PROJECT-GPT\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
class WikipediaSearchToolArgsSchema(BaseModel):

    query: str = Field(..., description="The search query.")


class DuckDuckGoSearchToolArgsSchema(BaseModel):

    query: str = Field(..., description="The search query.")


class DuckDuckGoScraperToolArgsSchema(BaseModel):

    url: str = Field(..., description="The URL to scrape.")


class SaveToFileToolArgsSchema(BaseModel):

    content: str = Field(..., description="The content to save.")


class WikipediaSearchTool(BaseTool):

    name = "WikipediaSearch"
    description = """
    Use this tool to perform searches on Wikipedia.
    It takes a query as an argument.
    Example query: "Artificial Intelligence"
    """
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query) -> Any:
        wikipedia_api = WikipediaAPIWrapper()
        wikipedia = WikipediaQueryRun(api_wrapper=wikipedia_api)
        return wikipedia.run(query)


class DuckDuckGoSearchTool(BaseTool):

    name = "DuckDuckGoSearch"
    description = """
    Use this tool to perform web searches using the DuckDuckGo search engine.
    It takes a query as an argument.
    Example query: "Latest technology news"
    """
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query) -> Any:
        search = DuckDuckGoSearchResults()
        return search.run(query)


class DuckDuckGoScraperTool(BaseTool):

    name = "DuckDuckGoScraper"
    description = """
    If you found the website link in DuckDuckGo,
    Use this to get the content of the link for my research.
    It takes a URL as an argument.
    """
    args_schema: Type[DuckDuckGoScraperToolArgsSchema] = DuckDuckGoScraperToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content.replace("\n", "")


class SaveToFileTool(BaseTool):

    name = "Save_to_File"
    description = """
    Use this tool to save the content as a .txt file.
    """
    args_schema: Type[SaveToFileToolArgsSchema] = SaveToFileToolArgsSchema

    def _run(self, content) -> Any:
        print(content)
        with open("research_results.txt", "w") as file:
            file.write(content)
        return "Research results saved to research_results.txt"

In [3]:
system_message = SystemMessage(
    content="""
        You are a research expert.

        Your task is to use Wikipedia or DuckDuckGo to gather comprehensive and accurate information about the query provided. 

        When you find a relevant website through DuckDuckGo, you must scrape the content from that website. Use this scraped content to thoroughly research and formulate a detailed answer to the question. 

        Combine information from Wikipedia, DuckDuckGo searches, and any relevant websites you find. Ensure that the final answer is well-organized and detailed, and include citations with links (URLs) for all sources used.

        Your research should be saved to a .txt file, and the content should match the detailed findings provided. Make sure to include all sources and relevant information.

        The information from Wikipedia must be included.

        Ensure that the final .txt file contains detailed information, all relevant sources, and citations.
        """
)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        DuckDuckGoScraperTool(),
        SaveToFileTool(),
    ],
    agent_kwargs={"system_message": system_message},
)

c:\Users\KIM\Desktop\PROJECT-GPT\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
query = "Research about the XZ backdoor"
results = agent.run(query)
print(results)

c:\Users\KIM\Desktop\PROJECT-GPT\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (pr

c:\Users\KIM\Desktop\PROJECT-GPT\env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:48: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: Description . Malicious code was discovered in the upstream tarballs of xz, starting with version 5.6.0. Through a series of complex obfuscations, the liblzma build process extracts a prebuilt object file from a disguised test file existing in the source code, which is then used to modify specific functions in the liblzma code., title: Cve-2024-3094 - Nvd, link: https://nvd.nist.gov/vuln/detail/CVE-2024-3094], [snippet: On March 28, 2024 a backdoor was identified in XZ Utils. This vulnerability, CVE-2024-3094 with a CVSS score of 10 is a result of a software supply chain compromise impacting versions 5.6.0 and 5.6.1 of XZ Utils. The U.S. Cybersecurity and Infrastructure Security Agency (CISA) has recommended organizations to downgrade to a previous non-compromised XZ Utils version., title: Microsoft FAQ and guidance for XZ Utils backdoor, link: https://techcommunity.microsoft.com/blog/vulnerability-management/microsoft-faq-and-guidance-for-xz-utils-backdoor/4101961], [snippet